# Extract RGB from PAD Columns

<font color='red'>Warning!</font> This notebook is a work in progress.

This notebook describes the process of extracting RGB colors from a PAD. Please remember to <font color='green'>change the settings</font> in the [User Inputs](#User-Inputs) section.

## Contents

- [User Inputs](#User-Inputs)
- [Imports](#Imports)
- [Explanation of Data Structure](#Explanation-of-Data-Structure)
- [Common Functions](#Common-Functions)
- [Load Images](#Load-Images)
- [Manual Box Placement](#Manual-Box-Placement)
- [Sample](#Sample)
- [Export to CSV](#Export-to-CSV)
- [Plot Color Changes Over Time](#Plot-Color-Changes-Over-Time)
- [Save Data as CSV](#Save-Data-as-CSV)
- [Rectification](#Rectification) <font color='red'>Work in progress!</font>

## User Inputs

These settings are used to help with the box placement/pixel sampling.

In [ ]:
PADDING_USED_ON_SIDES_OF_PLOTS = 15

Change these variables. This serves as an example.

In [ ]:
USER_INPUT = [
    {
        "directory": "C:\\your\\directory\\one",
        "title": "Name 1",
        "box_settings": {
            "box_width": 40,
            "box_height": 300,
            "num_boxes": 12,
            "offset_top": 1550,
            "offset_left": 515,
            "horizontal_spacer": 113
        },
        "lanes_to_sample": [
            "A",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    },
    {
        "directory": "C:\\your\\directory\\two",
        "title": "Name 2",
        "box_settings": {
            "box_width": 40,
            "box_height": 170,
            "num_boxes": 12,
            "offset_top": 1730,
            "offset_left": 590,
            "horizontal_spacer": 112
        },
        "lanes_to_sample": [
            "A",
            "B",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    },
    {
        "directory": "C:\\your\\directory\\three",
        "title": "Name 3",
        "box_settings": {
            "box_width": 40,
            "box_height": 150,
            "num_boxes": 12,
            "offset_top": 1680,
            "offset_left": 555,
            "horizontal_spacer": 114
        },
        "lanes_to_sample": [
            "A",
            "B",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    },
    {
        "directory": "C:\\your\\directory\\four",
        "title": "Name 4",
        "box_settings": {
            "box_width": 40,
            "box_height": 150,
            "num_boxes": 12,
            "offset_top": 1515,
            "offset_left": 500,
            "horizontal_spacer": 113
        },
        "lanes_to_sample": [
            "A",
            "B",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    },
    {
        "directory": "C:\\your\\directory\\five",
        "title": "Name 5",
        "box_settings": {
            "box_width": 40,
            "box_height": 150,
            "num_boxes": 12,
            "offset_top": 1600,
            "offset_left": 610,
            "horizontal_spacer": 110
        },
        "lanes_to_sample": [
            "A",
            "B",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    },
    {
        "directory": "C:\\your\\other\\directory\\six",
        "title": "Name 6",
        "box_settings": {
            "box_width": 40,
            "box_height": 150,
            "num_boxes": 12,
            "offset_top": 1420,
            "offset_left": 560,
            "horizontal_spacer": 110
        },
        "lanes_to_sample": [
            "A",
            "B",
            "C",
            "D",
            "E",
            "F",
            "G",
            "H",
            "I",
            "J",
            "K",
            "L"
        ]
    }
]

In [ ]:
OUTPUT_DIRECTORY = r"C:\\your\\output\\directory"

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import sys
from datetime import datetime

In [ ]:
try:
    import pad
except ModuleNotFoundError:
    sys.path.insert(1, r"..\src")
    import pad

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams['figure.max_open_warning'] = False

## Explanation of Data Structure

The user provides a single or several directories containing images. For each directory, a variety of data is read and generated from the image files. For each directory, the data structure looks like this.

```
{
    "name": "This is the string that the user assigns. It can be anything. It's used in plotting.",
    "path_to_dir": "some/path/to/directory",
    "image_filenames": [Pathlib object, Pathlib object, Pathlib object],
    "column_data": Python dictionary,
    "summary_data": Python dictionary,
    "column_data_frame": Pandas table,
    "summary_data_frame": Pandas table    
}
```

I actually went a step beyond that and made it more formal through a class. Might be excessive, but hey, at least we know where everything is now.

## Load Images

Create output directory if it doesn't exist. Setup output filenames.

In [ ]:
output_dir = pathlib.Path(OUTPUT_DIRECTORY)
if not output_dir.is_dir():
    os.mkdir(str(output_dir))
    print("Created: {}".format(str(output_dir)))
else:
    print("Already exists: {}".format(str(output_dir)))

Get input filenames and organize them. Note that right now this only reads JPG, but that's pretty easy to change.

In [ ]:
input_directory_objects = pad.load_multiple_directories(USER_INPUT, verbose=True)

## Sample

This code samples the directories.

In [ ]:
for directory_object in input_directory_objects:
    directory_object.column_data, directory_object.summary_data = pad.sample_directory_and_create_output_data(
        image_filenames=directory_object.image_filenames,
        output_directory=OUTPUT_DIRECTORY,
        box_settings=directory_object.box_settings,
        lanes_to_sample=directory_object.lanes_to_sample,
        verbose=False,
        draw=False
    )

## Export to CSV

In [ ]:
for directory_object in input_directory_objects:
    directory_object.column_data_frame = pad.create_column_data_frame(directory_object.column_data)
    directory_object.summary_data_frame = pad.create_summary_data_frame(directory_object.summary_data)

## Plot Color Changes Over Time

Just a cute little sample of what we have.

In [ ]:
input_directory_objects[0].summary_data_frame.head()

In [ ]:
for directory_object in input_directory_objects:
    pad.plot_colors_over_time(
        directory_object.summary_data_frame,
        title=directory_object.name,
        save=True,
        output_dir=output_dir,
        pad_sides=PADDING_USED_ON_SIDES_OF_PLOTS
    )
    pad.plot_inverted_colors_over_time(
        directory_object.summary_data_frame,
        title=directory_object.name,
        save=True,
        output_dir=output_dir,
        pad_sides=PADDING_USED_ON_SIDES_OF_PLOTS
    )
    pad.plot_grayscale_over_time(
        directory_object.summary_data_frame,
        title=directory_object.name,
        save=True,
        output_dir=output_dir,
        pad_sides=PADDING_USED_ON_SIDES_OF_PLOTS
    )
    pad.plot_inverted_grayscale_over_time(
        directory_object.summary_data_frame,
        title=directory_object.name,
        save=True,
        output_dir=output_dir,
        pad_sides=PADDING_USED_ON_SIDES_OF_PLOTS
    )

In [ ]:
pad.plot_many_inverted_grayscale_intensities_over_time(
    [x.summary_data_frame for x in input_directory_objects],
    [x.name for x in input_directory_objects],
    save=True,
    output_dir=output_dir
)

## Save Data as CSV

In [ ]:
for directory_object in input_directory_objects:
    directory_object.save_data_frames_to_csv(output_dir=OUTPUT_DIRECTORY)

## Rectification

In [ ]:
# image = cv2.imread(str(image_filenames[0]), 1)
# pad.plot_image(image, title="Original")
# edges, contours, hierarchy = rectification.find_contours(image, verbose=False)
# markers = rectification.select_markers(contours, hierarchy, verbose=True, very_verbose=False)

In [ ]:
# image = cv2.imread(r"C:\Users\ericc\projects\padtools\untracked\raw.jpg", 1)
# pad.plot_image(image, title="Original")
# edges, contours, hierarchy = rectification.find_contours(image, verbose=False)
# markers = rectification.select_markers(contours, hierarchy, verbose=True, very_verbose=False)